In [82]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
# from seleniumwire import webdriver as wbw
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  
# from selenium.webdriver.common.by import By
# from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display

In [9]:
def get_soup(url): # Retrieves soup from website using py requests module and 'person' headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://duckduckgo.com/'
        }
    response = requests.get(url, headers=headers)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page)
    else:
      print(f"Oops! Received status code {status}")
    return soup

In [10]:
def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  

In [270]:
def get_soup_driver_proxy(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    YOUR_API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    target_url = urllib.parse.quote(base_url, safe='~')

    URL = f"https://api.webscrapingapi.com/v1/?api_key={YOUR_API_KEY}&url=" + target_url

    driver_proxy.get(URL)
    time.sleep(2 + random.uniform(-1,1))

    # check if accept cookie button

    try: 
        cookie_button = driver_proxy.find_elements_by_xpath('//button[@class="confirm button radius margin-small secondary right"]')
        cookie_button[0].click()
        time.sleep(3 + random.uniform(-1,0))
#         driver_proxy.get(URL)

    except:
        pass
    
    
    r = driver_proxy.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  
#     return r

In [117]:
def load_webpage_proxy(base_url):
    YOUR_API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    target_url = urllib.parse.quote(base_url, safe='~')
    URL = f"https://api.webscrapingapi.com/v1/?api_key={YOUR_API_KEY}&url=" + target_url
    driver_proxy.get(URL)

In [112]:
def search_overdrive(title): # Searches overdrive for a given title. Returns a df of urls and media type.
    base_url = 'https://nypl.overdrive.com/search?query='
    title_url = urllib.parse.quote(title, safe='~')
    soup = get_soup_driver_proxy(base_url + title_url)
    list_results = []
    
    try:
        soup.find('h1', id = 'noresults').text

        results = {}
        results['title'] = title
        results['overdrive_url'] = 'title not found'
        results['media'] = 'title not found'
        list_results += [results]
        
    except AttributeError:


        title_urls = soup.find_all('h3', class_='title-name')
        title_urls = [url.find('a').get('href') for url in title_urls]
        media = soup.find_all(class_='title-format-badge')
        media = [item.text for item in media]
        
        for i in range(len(media)):
            if i > 1: break # limit search results to 2. Most likely OD will return too many false results.
            results = {}
            results['title'] = title
            results['overdrive_url'] = title_urls[i]
            results['media'] = media[i]
            list_results += [results]
    return pd.DataFrame(list_results)   

In [285]:
'''Takes in a df for a single title containing either audio, ebook, or both.
    Extracts hold info from each url in the url column and adds to the df to 
    give it hold info.'''

def get_holds_info(df):
    base_url = 'https://nypl.overdrive.com'

#     # add columns that this function will fill in    
    new_columns = ['Overdrive title', 'Wait time', 'Library copies', 'People waiting in total', 
                   'People waiting per copy', 'Genres', 'overdrive_author']
#     for i in range(len(new_columns)):
#         df[new_columns[i]] = np.nan
  
    total_sub_df = pd.DataFrame(columns = new_columns)
    
    if list(df.overdrive_url)[0] != 'title not found':
#         driver = webdriver.Chrome()

        for item_url in df.overdrive_url:
            time.sleep(3 + random.uniform(-1,1))
            # load item page in Chrome
            load_webpage_proxy(base_url + item_url)
            time.sleep(10 + random.uniform(-1,1))
            
            # check for a holds_info button. If there, click it.
            try:
                
                try:
                    # get author
                    try: 
                        cookie_button = driver_proxy.find_elements_by_xpath('//button[@class="confirm button radius margin-small secondary right"]')
                        cookie_button[0].click()
                        time.sleep(2 + random.uniform(-1,0))
                #         driver_proxy.get(URL)

                    except Exception as e:
#                         print('didn\'t see cookie button')
#                         print(e)
                        pass
                        
                        
                    soup = bs(driver_proxy.page_source, 'html.parser')
#                     print(soup.find('a', class_='TitleDetailsHeading-creatorLink').text)
                    overdrive_author = soup.find('a', class_='TitleDetailsHeading-creatorLink').text
                    
                    
                    
                    
                except Exception as e:
                    overdrive_author = -1
                    print(e)

#                 print(overdrive_author)
                
                # click hold_time_info button
                buttons = driver_proxy.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
                buttons[0].click()
                time.sleep(3 + random.uniform(-1,1))

                # grab html and parse into soup
                soup = bs(driver_proxy.page_source, 'html.parser') 
                

                # extract hold_info like wait-time, number of people waiting, etc.
                holds_info_soup = soup.find_all('p', class_='detail')
                holds_info = [info.text.split(':') for info in holds_info_soup]

                # remove extra spaces
                list_results = []
                for row in holds_info:
                    row[:] = [info.strip() for info in row]

                # convert result into sub_df
                holds_info = np.array(holds_info)
                sub_df = pd.DataFrame(holds_info.T, columns = holds_info[:,0]) ### sub_df might only be partially formed         
                sub_df.drop([0], inplace=True)

                # if holds_info contains incomplete holds_info, raise Error to remove data point
#                 print(sub_df.columns)
#                 print(list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'])
                if list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy']:
                    raise Exception('column names don\'t match')
        
#                 print('checked for holds_info button')
#                 display(sub_df)
        
        
            # if no holds_info button, it may be because copies are available. Check.
            except:
#                 print('apparently there was an error checking for holds_button button. maybe missing col names or no holds button')
                # grab html and parse into soup
                soup = bs(driver_proxy.page_source, 'html.parser')
                
                # yes copies are available and therefore no holds_info button
                try:
                    # there is no holds_info button so check for borrow button
                    if soup.find('button', class_='TitleActionButton').text == 'Borrow':
                        copies_avail = soup.find('button', class_='TitleActionButton').parent.parent.parent.parent.find(class_='availabilityText').text.strip().split()
                        if copies_avail[0] == 'Always':
                            holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                     [0, 'Always available', 'Unknown', 0]])    
                        else:    
                            holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                     [0, copies_avail[2], str(-int(copies_avail[0])), 0]])
#                         print('there is a borrow button')
                        sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                        sub_df.drop([0], inplace=True)
                    
                    else:
#                         print('there is no borrow button')
                        raise Exception('no borrow button')
                
                # no copies available and no holds_info button
                except:
#                     print('no copies available and no holds_info button')
                    holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                 ['info unavailable', 'info unavailable', 'info unavailable', 'info unavailable']])
                
                
                
                    sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                    sub_df.drop([0], inplace=True)

            # insert list of genres into sub_df
            genres = soup.find_all('a', class_='TitleDetailsSidebar-subjectButton')
            genres = [genre.text for genre in genres]
            sub_df['Genres'] = [genres]

            # insert Overdrive title into sub_df
            sub_df['Overdrive title'] = soup.find('h1').text
            
            sub_df['overdrive_author'] = overdrive_author

            # add the results to the total_sub_df
            total_sub_df = pd.concat([total_sub_df, sub_df], axis=0)

        # reset index because += adds the wrong index value. You get [1,1] instead of [0,1] and it won't concat right.
#         driver.quit()
    else:
        sub_df = pd.DataFrame(columns = new_columns)
        sub_df.loc[0] = np.array(['title not found']*len(new_columns))
        total_sub_df = pd.concat([total_sub_df, sub_df], axis=0)
        
    total_sub_df.reset_index(inplace=True, drop=True)

    
    
    return pd.concat([df, total_sub_df], axis=1)

In [24]:
def scrape_goodreads(url): # returns a df containing titles, ratings, urls, date published
    soup = get_soup(url)
    list_results = []
    
    # get list of titles
    soup_titles = soup.find_all('a', class_='bookTitle')
    titles = [title.text.strip() for title in soup_titles]
    
    # get authors
    authors = soup.find_all('a', class_='authorName')
    authors = [author.text for author in authors]
    
    # get list of ratings 
    stars = soup.find_all(class_='minirating')
    stars = [star.text.split('—') for star in stars]

    # some stars have messed up title starting with a string. Clean this up.
    for star in stars:
        s1 = star[0].strip()
        star[0] = star[0].strip()
        m = re.search(r"\d", s1)
        index = m.start()
        if index:
            star[0] = star[0][index:]

        
    # clean up extra white spaces   
    cleaned_stars = []
    for line in stars:
        transfer = []
        for item in line:
            transfer += [item.strip()]
        cleaned_stars += [transfer]

    
    
    # get year published
#     dates = soup.find_all(class_='minirating')
#     dates = [date.parent.text.split()[-1] for date in dates]
    
    # get url for GR page for specific title
    urls = [url.get('href') for url in soup_titles]
    
    #assemble list of dicts for each title
    for i in range(len(titles)):
        results = {}
        title = titles[i]
        results['title'] = title
        results['author'] = authors[i]
        
        if cleaned_stars[i][0][0].isnumeric():       
            results['stars'] = cleaned_stars[i][0]
        else: results['stars'] = 'error'
        results['number of ratings'] = cleaned_stars[i][1]
#         results['year published'] = dates[i]
        results['url'] = urls[i]
        left = '(' in titles[i]
        right = ')' in titles[i]
        pound = '#' in titles[i]
        if left and right and pound:
            results['is_series'] = True
            
            # clean up title
            pound_ind = title.index('#')
            right_ind = title.index(')', pound_ind)
            title = title[::-1]
            left_ind = title.index('(', title.index('#'))
            left_ind = len(title) - left_ind - 1
            title = title[::-1]
            title = title[:left_ind] + title[right_ind + 1:]
            title = title.strip()
            results['title'] = title

        else: results['is_series'] = False


        list_results += [results]

    # convert list of dicts into df
    list_results = pd.DataFrame(list_results)
    
    # clean and return df
    return list_results

    



In [14]:
def clean_GR(df): # converts stars, # ratings, year to floats or ints.
    df['stars'] = df.stars.str.split().str[0].astype('float')
    df['number of ratings'] = df['number of ratings'].str.split().str[0].str.replace(',', '').astype('int')
    df['year published'] = df['year published'].astype('int')
    
    return df

In [149]:
def get_page_price_date_movie(title, author):
    base_url = 'https://www.google.com/search?q='
    title_url = urllib.parse.quote(title + ' book', safe='~')
    
    # do google search for book and get price
    soup = get_soup_driver(base_url + title_url)
    try:
        price = float(soup.find(text = 'Kindle Store').next.text.split()[0].replace('$', ''))
        price = float(price)
    except: 
        try: 
            price = float(soup.find(text = 'Barnes & Noble').next.text.split()[0].replace('$', ''))
            price = float(price)
        except:
            try:
                price = soup.find(text=re.compile('\$[0-9]+'))[1:]
                price = float(price)
            except:
                price = -1
    
    try:
#         print('try1')
        year_published = soup.find(text=re.compile('Originally published')).parent.parent.parent.find(class_='z8gr9e').text
    except:
        year_published = 0
        
    # get page count from a second google search
    title_url = urllib.parse.quote(title + ' page count', safe='~')    
    time.sleep(2 + random.uniform(-1,1))


    
    # use selenium because page number doesn't always come up using requests module.
    soup = get_soup_driver(base_url + title_url)

    if not year_published:
#         print('try2')
        try:
#             print(soup.find(text=re.compile('Originally published')).parent.parent.parent.find(class_='z8gr9e').text)
            year_published = soup.find(text=re.compile('Originally published')).parent.parent.parent.find(class_='z8gr9e').text
        except Exception as e:
            year_published = str(e)
            
    try: 
        page_num = int(soup.find('div', class_='IZ6rdc').text)
#         print('try 1')
    except: 
        try: 
#             print('try2')
            pages = soup.find('a', text='Page count')
            page_num = pages.parent.parent.parent.find(class_='z8gr9e').text
        except Exception as e:
            page_num = str(e)

    # try price again if necessary
    if price == -1:
        try:
            price = float(soup.find(text = 'Kindle Store').next.text.split()[0].replace('$', ''))
            price = float(price)
        except: 
            try: 
                price = float(soup.find(text = 'Barnes & Noble').next.text.split()[0].replace('$', ''))
                price = float(price)
            except:
                try:
                    price = soup.find(text=re.compile('\$[0-9]+'))[1:]
                    price = float(price)
                except Exception as e:
                    price = -1
    
    
    
    
    
    # get movie info
    
    movie_info = search_imdb(title)
    
    if movie_info[0]:
        movie_title = movie_info[1]['movie title']
        movie_year = movie_info[1]['movie date']
    else:
        movie_title = 'N/A'
        movie_year = 'N/A'  
    
    result = pd.DataFrame({'year_published':[year_published], 'title':[title], 'price':[price], 
                           'number of pages':[page_num], 'is_movie':movie_info[0], 'movie title': movie_title,
                           'movie year':movie_year})
    
    return result

In [160]:
def get_title_info_audio_ebook(df): # takes in df from goodreads, adds page count, price, holds info
    global driver, driver_proxy
    driver = webdriver.Chrome()
    driver_proxy = initiate_wbw()
#     print('ding')
    imported_df = restart_df('Figures/good_reads_df.csv')
    row_start = imported_df[0]

    if row_start == 0:    
        df[['year_published', 'price', 'number of pages', 'is_movie', 'movie title', 'movie year', 'overdrive_url', 
            'media', 'Overdrive title', 'Wait time', 'Library copies', 'People waiting in total', 
            'People waiting per copy', 'Genres']] = np.nan

#         df.insert(0, 'id_number', df.index)
    # shuffle df to avoid patterns and chances of blocked webscraper
#     df.sample(frac=1)
    else:
        df = imported_df[1]

#     list(df.id_number)[list(df.id_number).index(row_start):]
#     print(list(df.id_number))
#     print(row_start in list(df.id_number))
    indices = list(df.id_number)[list(df.id_number).index(row_start):]
    titles = list(df.title)[list(df.id_number).index(row_start):]
    authors = list(df.author)[list(df.id_number).index(row_start):]
    counter = 0
     
#     display(df)
    
    for index, title, author in list(zip(indices, titles, authors)):
        # get page count and price and add to df
        page_price_date_movie = get_page_price_date_movie(title, author)
        index = list(df.id_number).index(index)
        df.loc[index, 'year_published'] = page_price_date_movie.loc[0, 'year_published']
        df.loc[index, 'price'] = page_price_date_movie.loc[0, 'price']
        df.loc[index, 'number of pages'] = page_price_date_movie.loc[0, 'number of pages']
        df.loc[index, 'is_movie'] = page_price_date_movie.loc[0, 'is_movie']
        df.loc[index, 'movie title'] = page_price_date_movie.loc[0, 'movie title']
        df.loc[index, 'movie year'] = page_price_date_movie.loc[0, 'movie year']
        
#         print(df)
#         df = df.merge(page_price, how='left', on='title')
#         display(df)
        # get hold info
        OD_results = search_overdrive(title)
        holds_info_df = get_holds_info(OD_results)
        df = merge_holds_info(df, holds_info_df)
#         print(holds_info_df)
#         print(df)
#         df.loc[index, 'Overdrive title'] =  holds_info_df.loc[0, 'Overdrive title']
#         df.loc[index, 'Wait time'] =  holds_info_df.loc[0, 'Wait time']
#         df.loc[index, 'Library copies'] =  holds_info_df.loc[0, 'Library copies']
#         df.loc[index, 'People waiting in total'] =  holds_info_df.loc[0, 'People waiting in total']
#         df.loc[index, 'People waiting per copy'] =  holds_info_df.loc[0, 'People waiting per copy']
#         df.loc[index, 'overdrive_url'] =  holds_info_df.loc[0, 'overdrive_url']
#         df.loc[index, 'media'] =  holds_info_df.loc[0, 'media']
# #         print(df)
#         df.loc[index, 'Genres'] = ', '.join(holds_info_df.loc[0, 'Genres'])
        
        df.to_csv('Figures/good_reads_df.csv')
        time.sleep(10 + random.uniform(-1,1))

#         df = merge_holds_info(df, holds_info_df)
    driver.quit()
    driver_proxy.quit()
    return df
        
        

In [17]:
def merge_holds_info(df, holds_info):

    # if this is the first time merging, new columns will be added to the df so no problems with column names
    if 'overdrive_url' not in list(df.columns):
        df = df.merge(holds_info, how='left', on='title')
#         print('not in')
    # if this column has already merged, there will be other columns already present. Not doing anything
#     will cause duplicate columns upon merging.
    else:
        # rename columns to stop pd from changing duplicate names (e.g. url to url_x)
        
        #capture original holds_info_columns (overdrive_url, etc.) to later rename them back below
        holds_info_columns = holds_info.columns
        
        # make new temp column names for everything after 'title'
        new_holds_info_columns = [holds_info_columns[0]] + list(range(len(holds_info_columns)-1))
        
        # set column names to new column names from above
        holds_info.set_axis(new_holds_info_columns, axis=1, inplace=True)
        
        # merge dfs to allow duplicate titles (one audio, one book)to be populated
        # this will create a duplicate column of hold info but labeled 0,1,2,3 etc.
        df = df.merge(holds_info, how='left', on='title')   
        
        # after merge, create a copy of columns to be merged from both dfs
        df_merge = df.iloc[:, list(df.columns).index('overdrive_url'):list(df.columns).index(0)].copy()      
        
        # create a copy of columns to be merged labeled with 0,1,2,etc and rename them back to original names
        df_merge_new_info = df.iloc[:, list(df.columns).index(0):].copy()
#         print(df_merge_new_info)
        # rename them back
        df_merge_new_info.columns = holds_info_columns[1:]
        
        # now merge
        merged_df = df_merge.combine_first(df_merge_new_info)
        
        # replace columns in original df with merged_df and delete the extra columns at the end
        df.iloc[:, list(df.columns).index('overdrive_url'):list(df.columns).index(0)] = merged_df
        df = df.iloc[:,:list(df.columns).index(0)]
        
    return df

In [18]:
def restart_df(file): # specify file location/name of a df, find index of where to pick up and return partially completed df
    try: 
        df = pd.read_csv(file, index_col=0)
    except FileNotFoundError:
        return 0, 0
    
    return df[df.price.isna()].iloc[0,0], df
    
    

In [32]:
def search_imdb(title): # get list of movies from IMDB with the title
    title = urllib.parse.quote(title, safe='~')
    url = f'https://www.imdb.com/search/title/?title={title}&title_type=feature'
    soup = get_soup(url)
    titles = soup.find_all('h3')
    result = {}
#     titles = [title.text.strip() for title in soup.find_all('h3')]
#     print(titles)
    if len(titles) > 1:
        result['movie title'] = titles[0].find('a').text.strip()
        result['movie date'] = titles[0].find(class_='lister-item-year').text[1:-1]
    else:
        result['movie title'] = False
        result['movie date'] = False

    return len(titles) > 1, result
        


In [93]:
# test = most_read_GR.copy().iloc[34:37].reset_index(drop=True)
test = most_read_GR.copy().iloc[:10].reset_index(drop=True)

In [94]:
test

,title,author,stars,number of ratings,url,is_series
0,The Hunger Games,Suzanne Collins,4.32 avg rating,"7,168,601 ratings",/book/show/2767052-the-hunger-games,True
1,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.50 avg rating,"2,848,336 ratings",/book/show/2.Harry_Potter_and_the_Order_of_the...,True
2,To Kill a Mockingbird,Harper Lee,4.27 avg rating,"5,108,054 ratings",/book/show/2657.To_Kill_a_Mockingbird,False
3,Pride and Prejudice,Jane Austen,4.27 avg rating,"3,469,232 ratings",/book/show/1885.Pride_and_Prejudice,False
4,Twilight,Stephenie Meyer,3.62 avg rating,"5,579,253 ratings",/book/show/41865.Twilight,True
5,The Book Thief,Markus Zusak,4.38 avg rating,"2,105,258 ratings",/book/show/19063.The_Book_Thief,False
6,Animal Farm,George Orwell,3.97 avg rating,"3,157,234 ratings",/book/show/170448.Animal_Farm,False
7,The Chronicles of Narnia,C.S. Lewis,4.25 avg rating,"579,252 ratings",/book/show/11127.The_Chronicles_of_Narnia,True
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.60 avg rating,"120,247 ratings",/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set,False
9,The Fault in Our Stars,John Green,4.17 avg rating,"4,155,620 ratings",/book/show/11870085-the-fault-in-our-stars,False


In [115]:
master_GR_df = pd.read_csv('Figures/master_GR_df.csv', index_col=0)

In [ ]:
def run_scraper(df):
    try: 
        get_title_info_audio_ebook(df)
    except:
        driver.quit()
        driver_proxy.quit()
        time.sleep(10)
        run_scraper(df)

In [ ]:
run_scraper(master_GR_df)

In [299]:
file = 'Figures/good_reads_df.csv'
df = pd.read_csv(file, index_col=0)

In [292]:
get_title_info_audio_ebook(df)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59398): Max retries exceeded with url: /session/ff8b2b2884b3cc92f7a9ab9514e95186/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16af3cdc0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [308]:
def new_scrape(df, error_counter=0):
    import datetime
    error_counter = error_counter
    try: get_title_info_audio_ebook(df)
    except:
        print(datetime.datetime.now())
        time.sleep(180 + random.uniform(-2,2))
        error_counter += 1
        driver_proxy.quit()
        driver.quit()
        if error_counter > 10:
            return 'Terminated, check for errors'
        new_scrape(df, error_counter)

In [ ]:


datetime.datetime.now()

In [1257]:
driver = webdriver.Chrome()
soup = get_soup_driver('https://nypl.overdrive.com/media/6070432')

buttons = driver.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
buttons[0].click()
time.sleep(2 + random.uniform(-1,1))

# grab html and parse into soup
soup = bs(driver.page_source, 'html.parser') 

# extract hold_info like wait-time, number of people waiting, etc.
holds_info_soup = soup.find_all('p', class_='detail')
holds_info = [info.text.split(':') for info in holds_info_soup]

# remove extra spaces
list_results = []
for row in holds_info:
    row[:] = [info.strip() for info in row]

# convert result into sub_df
holds_info = np.array(holds_info)
sub_df = pd.DataFrame(holds_info.T, columns = holds_info[:,0]) 
sub_df.drop([0], inplace=True)

In [1260]:
driver = webdriver.Chrome()
soup = get_soup_driver('https://nypl.overdrive.com/media/791128')

buttons = driver.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
buttons[0].click()
time.sleep(2 + random.uniform(-1,1))

# grab html and parse into soup
soup = bs(driver.page_source, 'html.parser') 

# extract hold_info like wait-time, number of people waiting, etc.
holds_info_soup = soup.find_all('p', class_='detail')
holds_info = [info.text.split(':') for info in holds_info_soup]

# remove extra spaces
list_results = []
for row in holds_info:
    row[:] = [info.strip() for info in row]

# convert result into sub_df
holds_info = np.array(holds_info)
sub_df = pd.DataFrame(holds_info.T, columns = holds_info[:,0]) 
sub_df.drop([0], inplace=True)

In [153]:
def initiate_wbw(): # launches headless browser selenium
    options = Options()
    options.headless = False
    return webdriver.Chrome(options=options)

In [155]:
# driver = initiate_wbw()
# driver.quit()

In [283]:
df.drop_duplicates(subset=['url', 'overdrive_url']).to_csv('Figures/test.csv')

In [223]:
file2 = 'Figures/get_author_OD.csv'
get_author_OD = pd.read_csv(file2, index_col=0)

In [210]:
print(df.columns)
print(get_author_OD.columns)

Index(['id_number', 'title', 'author', 'stars', 'number of ratings', 'url',
       'is_series', 'year_published', 'price', 'number of pages', 'is_movie',
       'movie title', 'movie year', 'overdrive_url', 'media',
       'Overdrive title', 'Wait time', 'Library copies',
       'People waiting in total', 'People waiting per copy', 'Genres'],
      dtype='object')
Index(['id_number', 'title', 'author', 'stars', 'number of ratings', 'url',
       'is_series', 'year_published', 'price', 'number of pages', 'is_movie',
       'movie title', 'movie year', 'overdrive_url', 'media',
       'Overdrive title', 'Wait time', 'Library copies',
       'People waiting in total', 'People waiting per copy', 'Genres',
       'overdrive_author'],
      dtype='object')


In [213]:
get_author_OD.loc[:,['id_number','overdrive_url', 'overdrive_author']]

,id_number,overdrive_url,overdrive_author
0,0,/media/3946821,Elie Wiesel
1,0,/media/90613,Elie Wiesel
2,1,/media/283610,Ellen Raskin
3,1,/media/3326221,Ellen Raskin
4,2,/media/5408082,Nick Spencer
...,...,...,...
1741,995,NaN,NaN
1742,996,NaN,NaN
1743,997,NaN,NaN
1744,998,NaN,NaN


In [216]:
test = df.merge(get_author_OD.loc[:,['id_number','overdrive_url', 'overdrive_author']], on=['id_number','overdrive_url'], how='left')

In [224]:
get_author_OD[get_author_OD['Library copies'] == 'info unavailable'].iloc[0].name

72

In [225]:
get_author_OD[(get_author_OD['Library copies'] == 'info unavailable') | (get_author_OD.overdrive_author.isna())]

,id_number,title,author,stars,number of ratings,url,is_series,year_published,price,number of pages,...,movie year,overdrive_url,media,Overdrive title,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author
72,39.0,If on a Winter's Night a Traveler,Italo Calvino,4.04 avg rating,"82,880 ratings",/book/show/374233.If_on_a_Winter_s_Night_a_Tra...,False,"June 2, 1979",13.59,272,...,NaN,/media/3783541,Audiobook,If On a Winter's Night a Traveler,info unavailable,info unavailable,info unavailable,info unavailable,"['Classic Literature', 'Fiction']",Italo Calvino
105,56.0,A Court of Wings and Ruin,Sarah J. Maas,4.41 avg rating,"472,407 ratings",/book/show/23766634-a-court-of-wings-and-ruin,True,"May 2, 2017",9.99,736,...,NaN,/media/3255025,ebook,A Court of Wings and Ruin,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Romance', 'Young Adult Fiction']",Sarah J. Maas
110,58.0,The History of Love,Nicole Krauss,3.91 avg rating,"130,457 ratings",/book/show/3867.The_History_of_Love,False,2005,8.74,'NoneType' object has no attribute 'parent',...,2016,/media/2852758,Audiobook,The History of Love,info unavailable,info unavailable,info unavailable,info unavailable,"['Fiction', 'Literature']",-1
116,61.0,The Night Before Christmas,Clement C. Moore,4.37 avg rating,"115,916 ratings",/book/show/196970.The_Night_Before_Christmas,False,Board Book,0.99,32,...,2019,/media/640168,ebook,The Night Before Christmas,info unavailable,info unavailable,info unavailable,info unavailable,"['Classic Literature', 'Juvenile Fiction', 'Ju...",Clement C. Moore
118,62.0,Possession,A.S. Byatt,3.87 avg rating,"80,389 ratings",/book/show/41219.Possession,False,1990,13.99,'NoneType' object has no attribute 'parent',...,1981,/media/76307,Audiobook,Possession,info unavailable,info unavailable,info unavailable,info unavailable,"['Fiction', 'Literature', 'Romance']",A.S. Byatt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919,997.0,The Complete Stories and Poems,Edgar Allan Poe,4.37 avg rating,"233,410 ratings",/book/show/23919.The_Complete_Stories_and_Poems,False,1966,9.99,850,...,NaN,/media/3302344,ebook,"Drafts, Fragments, and Poems",0,2,-2,0,"['Fiction', 'Poetry']",NaN
1920,998.0,Interview with the Vampire,Anne Rice,4.00 avg rating,"526,137 ratings",/book/show/43763.Interview_with_the_Vampire,True,"May 5, 1976",8.99,'NoneType' object has no attribute 'parent',...,1994,/media/451325,Audiobook,Interview with the Vampire,About 10 weeks,8,38,5,"['Fantasy', 'Fiction', 'Horror', 'Thriller']",NaN
1921,998.0,Interview with the Vampire,Anne Rice,4.00 avg rating,"526,137 ratings",/book/show/43763.Interview_with_the_Vampire,True,"May 5, 1976",8.99,'NoneType' object has no attribute 'parent',...,1994,/media/480415,ebook,Interview with the Vampire,About 6 weeks,10,25,3,"['Fantasy', 'Fiction', 'Horror', 'Thriller']",NaN
1922,999.0,A Prayer for Owen Meany,John Irving,4.23 avg rating,"298,820 ratings",/book/show/4473.A_Prayer_for_Owen_Meany,False,March 1989,8.99,617,...,NaN,/media/784063,ebook,A Prayer for Owen Meany,About 5 weeks,7,14,2,"['Classic Literature', 'Fiction', 'Literature']",NaN


In [217]:
# test = df.drop_duplicates(subset=['url','overdrive_url'])
test.to_csv('Figures/test.csv')

In [221]:
get_author_OD[get_author_OD.overdrive_author == '-1']

,id_number,title,author,stars,number of ratings,url,is_series,year_published,price,number of pages,...,movie year,overdrive_url,media,Overdrive title,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author
110,58,The History of Love,Nicole Krauss,3.91 avg rating,"130,457 ratings",/book/show/3867.The_History_of_Love,False,2005,8.74,'NoneType' object has no attribute 'parent',...,2016,/media/2852758,Audiobook,The History of Love,info unavailable,info unavailable,info unavailable,info unavailable,"['Fiction', 'Literature']",-1
139,74,Oryx and Crake,Margaret Atwood,4.00 avg rating,"243,294 ratings",/book/show/46756.Oryx_and_Crake,True,May 2003,9.99,400,...,NaN,/media/133417,Audiobook,Oryx and Crake,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Fiction', 'Literature', 'Science ...",-1
148,78,Assassin's Apprentice,Robin Hobb,4.16 avg rating,"267,106 ratings",/book/show/77197.Assassin_s_Apprentice,True,"April 1, 1995",7.99,448,...,NaN,/media/23231,ebook,Assassin's Apprentice,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Fiction']",-1
159,84,The Magus,John Fowles,4.05 avg rating,"53,402 ratings",/book/show/16286.The_Magus,False,1965,'NoneType' object is not subscriptable,'NoneType' object has no attribute 'parent',...,1968,/media/1055187,Audiobook,The Magus,info unavailable,info unavailable,info unavailable,info unavailable,"['Classic Literature', 'Fiction']",-1
171,90,Cosmos,Carl Sagan,4.37 avg rating,"130,653 ratings",/book/show/55030.Cosmos,False,1980,12.99,365,...,2019,/media/564597,ebook,Cosmos,info unavailable,info unavailable,info unavailable,info unavailable,"['Science', 'Nonfiction']",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,722,زندگی مه آلود پریا,سید مرتضی مصطفوی,3.89 avg rating,961 ratings,/book/show/40937505,False,'NoneType' object has no attribute 'parent','NoneType' object is not subscriptable,'NoneType' object has no attribute 'parent',...,NaN,title not found,title not found,title not found,title not found,title not found,title not found,title not found,title not found,-1
1454,764,القرآن الكريم,Anonymous,4.36 avg rating,"61,620 ratings",/book/show/646462._,False,'NoneType' object has no attribute 'parent','NoneType' object is not subscriptable,'NoneType' object has no attribute 'parent',...,NaN,title not found,title not found,title not found,title not found,title not found,title not found,title not found,title not found,-1
1457,766,The Complete Maus,Art Spiegelman,4.52 avg rating,"167,976 ratings",/book/show/15195.The_Complete_Maus,False,1980,'NoneType' object is not subscriptable,'NoneType' object has no attribute 'parent',...,NaN,title not found,title not found,title not found,title not found,title not found,title not found,title not found,title not found,-1
1476,776,The Tell-Tale Heart and Other Writings,Edgar Allan Poe,4.18 avg rating,"230,404 ratings",/book/show/391729.The_Tell_Tale_Heart_and_Othe...,False,1948,4.61,68,...,NaN,title not found,title not found,title not found,title not found,title not found,title not found,title not found,title not found,-1


In [227]:
file3 = 'Figures/master_GR_df_2.csv'
df2 = pd.read_csv(file3, index_col=0)
df2

,title,author,author_url,stars,number of ratings,url,is_series
0,On the Island,Tracey Garvis Graves,https://www.goodreads.com/author/show/5193034....,4.11 avg rating,"79,689 ratings",/book/show/15505346-on-the-island,True
1,This Side of Paradise,F. Scott Fitzgerald,https://www.goodreads.com/author/show/3190.F_S...,3.65 avg rating,"68,250 ratings",/book/show/46165.This_Side_of_Paradise,False
2,War and Remembrance,Herman Wouk,https://www.goodreads.com/author/show/9020.Her...,4.40 avg rating,"34,302 ratings",/book/show/42986.War_and_Remembrance,True
3,Defending Jacob,William Landay,https://www.goodreads.com/author/show/218843.W...,3.98 avg rating,"286,315 ratings",/book/show/11367726-defending-jacob,False
4,Dead in the Family,Charlaine Harris,https://www.goodreads.com/author/show/17061.Ch...,3.88 avg rating,"160,468 ratings",/book/show/7091488-dead-in-the-family,True
...,...,...,...,...,...,...,...
1095,Keeper of the Lost Cities,Shannon Messenger,https://www.goodreads.com/author/show/5265998....,4.44 avg rating,"45,949 ratings",/book/show/12842828-keeper-of-the-lost-cities,True
1096,The Rescue,Nicholas Sparks,https://www.goodreads.com/author/show/2345.Nic...,4.11 avg rating,"175,884 ratings",/book/show/3462.The_Rescue,False
1097,Fallen Too Far,Abbi Glines,https://www.goodreads.com/author/show/4872191....,4.17 avg rating,"167,610 ratings",/book/show/16070903-fallen-too-far,True
1098,Pretty Little Liars,Sara Shepard,https://www.goodreads.com/author/show/93970.Sa...,3.86 avg rating,"195,879 ratings",/book/show/162085.Pretty_Little_Liars,True


In [247]:
# test = df.append(df2).reset_index(drop=True)
test.to_csv('Figures/good_reads_df.csv')

In [ ]:
so